In [1]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as HTML
from IPython.display import HTML

from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-07-26 08:45:42.638311: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-07-26 08:45:42.638352: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
IMAGE_SIZE = 256
CHANNELS = 3
BATCH_SIZE = 32


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10
)

train_generator = train_datagen.flow_from_directory(
    directory='dataset/train',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
    )

Found 1506 images belonging to 3 classes.


In [4]:
val_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10
)

val_generator = val_datagen.flow_from_directory(
    directory='dataset/val',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
    )

Found 215 images belonging to 3 classes.


In [5]:
test_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10
)

test_generator = test_datagen.flow_from_directory(
    directory='dataset/test',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
    )

Found 431 images belonging to 3 classes.


In [6]:
train_generator.class_indices


{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}

In [7]:
class_names = list(train_generator.class_indices.keys())
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [8]:
for image_batch, batch_size in train_generator:
    print(image_batch[0])
    print(batch_size)
    break


[[[0.66351056 0.6556674  0.71056944]
  [0.6661626  0.6583195  0.71322143]
  [0.66881466 0.6609715  0.7158735 ]
  ...
  [0.7820682  0.77422506 0.82520545]
  [0.78823537 0.7803922  0.8313726 ]
  [0.78285396 0.7750108  0.8259912 ]]

 [[0.6505744  0.64273125 0.6976332 ]
  [0.65163517 0.64379203 0.698694  ]
  [0.652696   0.6448529  0.69975483]
  ...
  [0.78365946 0.7758163  0.8267967 ]
  [0.78823537 0.7803922  0.8313726 ]
  [0.78126276 0.7734196  0.8244    ]]

 [[0.65131444 0.6434713  0.69837326]
  [0.65025365 0.6424105  0.6973125 ]
  [0.6491928  0.6413497  0.69625163]
  ...
  [0.78525066 0.7774075  0.8283879 ]
  [0.78823537 0.7803922  0.8313726 ]
  [0.7796715  0.77182835 0.82280874]]

 ...

 [[0.52423877 0.5085525  0.56345445]
  [0.46320915 0.44752288 0.5024248 ]
  [0.50583935 0.49015307 0.54505503]
  ...
  [0.5693527  0.56150955 0.61248994]
  [0.57234335 0.5645002  0.6154806 ]
  [0.57658666 0.5687435  0.6197239 ]]

 [[0.5157522  0.5000659  0.5549679 ]
  [0.4690436  0.45335734 0.5082593 ]


## Building the Model

In [9]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = len(class_names)

model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.Conv2D(32, kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])


2025-07-26 08:45:45.196194: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2025-07-26 08:45:45.205094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-26 08:45:45.205356: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3050 Laptop GPU computeCapability: 8.6
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 178.84GiB/s
2025-07-26 08:45:45.205526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-07-26 08:45:45.205635: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dyna

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        3

In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    batch_size=BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    verbose=1,
    epochs=20
    )

2025-07-26 08:45:46.171176: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2025-07-26 08:45:46.171783: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2688000000 Hz


Epoch 1/20
47/47 [==============================] - 47s 968ms/step - loss: 0.8611 - accuracy: 0.5400 - val_loss: 0.6803 - val_accuracy: 0.6979
Epoch 2/20
47/47 [==============================] - 45s 952ms/step - loss: 0.4504 - accuracy: 0.8019 - val_loss: 0.4127 - val_accuracy: 0.8490
Epoch 3/20
47/47 [==============================] - 46s 978ms/step - loss: 0.3658 - accuracy: 0.8433 - val_loss: 0.3429 - val_accuracy: 0.8646
Epoch 4/20
47/47 [==============================] - 45s 952ms/step - loss: 0.3505 - accuracy: 0.8636 - val_loss: 0.4810 - val_accuracy: 0.7917
Epoch 5/20
47/47 [==============================] - 46s 970ms/step - loss: 0.2689 - accuracy: 0.8921 - val_loss: 0.3316 - val_accuracy: 0.8542
Epoch 6/20
47/47 [==============================] - 46s 959ms/step - loss: 0.3090 - accuracy: 0.8704 - val_loss: 0.3321 - val_accuracy: 0.8385
Epoch 7/20
47/47 [==============================] - 45s 956ms/step - loss: 0.2509 - accuracy: 0.8955 - val_loss: 0.3057 - val_accuracy: 0.8542

In [ ]:
scores = model.evaluate(test_generator)


NameError: name 'model' is not defined